In [ ]:
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
from glob import glob
from statsmodels.tsa.stattools import adfuller
from scipy import signal
matplotlib.rcParams['font.family'] = 'STIXGeneral'

In [ ]:
review_df = pd.read_csv('data.csv')
stock_df = pd.read_parquet("stock_data/financial_data.parquet")

In [ ]:
review_df['date'] = [pd.to_datetime(date) for date in review_df.date]
review_df['year'] = [date.year for date in review_df.date]

In [ ]:
def rolling_reviews(data, start_date, end_date, window_size=7, spill_over=3):

    end = pd.to_datetime(end_date)
    ratings = list()
    i = pd.to_datetime(start_date)
    j = i + dt.timedelta(days=window_size)

    while j < end:
        i_window = i - dt.timedelta(days=spill_over)
        j_window = j + dt.timedelta(days=spill_over)
        subset = data[(data.index >= i_window) & (data.index < j_window)]
        if (n := len(subset)) == 0:
            ratings.append((np.nan, n, i))
        else:
            avg_rating = np.mean(subset)
            ratings.append((avg_rating, n, i))

        i, j = j, j + dt.timedelta(days=window_size)

    return ratings


def rolling_stocks(data, start_date, end_date, window_size=7, spill_over=3):

    end = pd.to_datetime(end_date)
    ratings = list()
    i = pd.to_datetime(start_date)
    j = i + dt.timedelta(days=window_size)

    while j < end:
        i_window = i - dt.timedelta(days=spill_over)
        j_window = j + dt.timedelta(days=spill_over)
        subset = data[(data.index >= i_window) & (data.index < j_window)]
        if (n := len(subset)) == 0:
            ratings.append((np.nan, n, i))
        else:
            avg_rating = np.mean(subset)
            ratings.append((avg_rating, n, i))

        i, j = j, j + dt.timedelta(days=window_size)

    return ratings

# zi = (xi – min(x)) / (max(x) – min(x))
def scale(x, x_max, x_min):

    z = (x - x_min) / (x_max - x_min)

    return z

In [ ]:
def data_wrangling(stock_df, review_df, company, start_date, end_date, window_size, spill_over):
    """
    This implementation scales first and then creates the rolling average over week which is then plotted.
    This in turn makes it such that none of the weeks plotted are at the minimum for Glassdoor ratings (problem?).
    """

    if company=='Apple':
        comp_tag = 'AAPL'
    elif company=='Meta':
        comp_tag = 'META'
    elif company=='Google':
        comp_tag = 'GOOG'
    elif company=='Amazon':
        comp_tag = 'AMZN'
    elif company=='Microsoft':
        comp_tag = 'MSFT'
    elif company=='Nvidia':
        comp_tag = 'NVDA'
    elif company=='Tesla':
        comp_tag = 'TSLA'

    start, end = pd.to_datetime(start_date), pd.to_datetime(end_date)
    stocks = stock_df[comp_tag]
    stocks = stocks[(stocks.index >= start) & (stocks.index < end)]

    reviews = review_df[review_df["company"]==f"{company}"]
    reviews['rating'] = reviews.rating.apply(float)
    reviews['date'] = [pd.to_datetime(date) for date in reviews.date]
    reviews = reviews[(reviews.date >= start) & (reviews.date < end)]
    reviews = reviews.groupby('date').mean().fillna(np.mean(reviews))

    reviews = scale(reviews.rating, max(reviews.rating), min(reviews.rating))

    stocks = scale(stocks['Adj Close'], np.nanmax(stocks['Adj Close']), np.nanmin(stocks['Adj Close']))
    reviews_smoothed = rolling_reviews(reviews, start, end, window_size=window_size, spill_over=spill_over)
    stocks_smoothed = rolling_stocks(stocks, start, end, window_size=window_size, spill_over=spill_over)

    return reviews_smoothed, stocks_smoothed, stocks

In [ ]:
reviews_smoothed_apple, stocks_smoothed_apple, stocks_apple_nosmooth = data_wrangling(stock_df, review_df, 'Apple', '2020-01-01', '2022-10-01', 7, 2)
ratings_apple, _, rating_dates_apple = zip(*reviews_smoothed_apple)
stocks_apple, _, stock_dates_apple = zip(*stocks_smoothed_apple)

reviews_smoothed_amazon, stocks_smoothed_amazon, stocks_amazon_nosmooth = data_wrangling(stock_df, review_df, 'Amazon', '2020-01-01', '2022-10-01', 7, 2)
ratings_amazon, _, rating_dates_amazon = zip(*reviews_smoothed_amazon)
stocks_amazon, _, stock_dates_amazon = zip(*stocks_smoothed_amazon)

reviews_smoothed_meta, stocks_smoothed_meta, stocks_meta_nosmooth = data_wrangling(stock_df, review_df, 'Meta', '2020-01-01', '2022-10-01', 7, 2)
ratings_meta, _, rating_dates_meta = zip(*reviews_smoothed_meta)
stocks_meta, _, stock_dates_meta = zip(*stocks_smoothed_meta)

reviews_smoothed_google, stocks_smoothed_google, stocks_google_nosmooth = data_wrangling(stock_df, review_df, 'Google', '2020-01-01', '2022-10-01', 7, 2)
ratings_google, _, rating_dates_google = zip(*reviews_smoothed_google)
stocks_google, _, stock_dates_google = zip(*stocks_smoothed_google)

reviews_smoothed_microsoft, stocks_smoothed_microsoft, stocks_microsoft_nosmooth = data_wrangling(stock_df, review_df, 'Microsoft', '2020-01-01', '2022-10-01', 7, 2)
ratings_microsoft, _, rating_dates_microsoft = zip(*reviews_smoothed_microsoft)
stocks_microsoft, _, stock_dates_microsoft = zip(*stocks_smoothed_microsoft)

reviews_smoothed_nvidia, stocks_smoothed_nvidia, stocks_nvidia_nosmooth = data_wrangling(stock_df, review_df, 'Nvidia', '2020-01-01', '2022-10-01', 7, 2)
ratings_nvidia, _, rating_dates_nvidia = zip(*reviews_smoothed_nvidia)
stocks_nvidia, _, stock_dates_nvidia = zip(*stocks_smoothed_nvidia)

reviews_smoothed_tesla, stocks_smoothed_tesla, stocks_tesla_nosmooth = data_wrangling(stock_df, review_df, 'Tesla', '2020-01-01', '2022-10-01', 7, 2)
ratings_tesla, _, rating_dates_tesla = zip(*reviews_smoothed_tesla)
stocks_tesla, _, stock_dates_tesla = zip(*stocks_smoothed_tesla)

In [ ]:
def plot_series(company, ax, ratings, stocks, dates, stocks_df, with_legend=False, with_date=False):
    """
    Potentially add a color for the values imputed with the mean.
    """

    mu = np.nanmean(ratings)
    ratings = pd.Series(ratings).fillna(mu)

    axis = ax.set_title(company)
    axis = ax.set_facecolor("white")
    axis = ax.plot(dates, ratings, color="red", label='aggregated rating')
    axis = ax.plot(dates, stocks, color="blue", label='aggregated stock')
    axis = ax.plot(stocks_df.index, stocks_df, color="blue", alpha=.3, label='stock')
    if with_legend:
        axis = ax.legend(loc="upper left")

    return axis

In [ ]:
plt.style.use('bmh')

fig, axes = plt.subplots(7, 1, figsize=(10, 14))
fig.suptitle('Aggregated Review Ratings v. Stock Price by Week', weight='bold', fontsize=16)

fig.text(0.51, -.01, 'Date', ha='center', fontsize=14)
fig.text(-.01, 0.5, 'Scaled Avg. Review Rating & Stock Price', fontsize=14, va='center', rotation='vertical')

ax1, ax2, ax3, ax4, ax5, ax6, ax7 = axes

plot_series('Apple', ax1, ratings_apple, stocks_apple, rating_dates_apple, stocks_apple_nosmooth, with_legend=True)
plot_series('Amazon', ax2, ratings_amazon, stocks_amazon, rating_dates_amazon, stocks_amazon_nosmooth)
plot_series('Google', ax3, ratings_google, stocks_google, rating_dates_google, stocks_google_nosmooth)
plot_series('Meta', ax4, ratings_meta, stocks_meta, rating_dates_meta, stocks_meta_nosmooth)
plot_series('Microsoft', ax5, ratings_microsoft, stocks_microsoft, rating_dates_microsoft, stocks_microsoft_nosmooth)
plot_series('Nvidia', ax6, ratings_nvidia, stocks_nvidia, rating_dates_nvidia, stocks_nvidia_nosmooth)
plot_series('Tesla', ax7, ratings_tesla, stocks_tesla, rating_dates_tesla, stocks_tesla_nosmooth)

fig.tight_layout()

# Cross-Correlation

In [ ]:
def pre_analysis(review_df, stock_df, company, start_date, end_date):

    if company=='Apple':
        comp_tag = 'AAPL'
    elif company=='Meta':
        comp_tag = 'META'
    elif company=='Google':
        comp_tag = 'GOOG'
    elif company=='Amazon':
        comp_tag = 'AMZN'
    elif company=='Microsoft':
        comp_tag = 'MSFT'
    elif company=='Nvidia':
        comp_tag = 'NVDA'
    elif company=='Tesla':
        comp_tag = 'TSLA'
    
    start, end = pd.to_datetime(start_date), pd.to_datetime(end_date)

    reviews = review_df[review_df.company==company][['rating', 'date']]
    reviews['date'] = [pd.to_datetime(date) for date in reviews.date]
    reviews = reviews[(reviews.date >= start) & (reviews.date < end)]
    reviews['rating'] = reviews.rating.apply(float)
    reviews = reviews.groupby('date').mean()

    stocks = stock_df[comp_tag][['Adj Close']].dropna().rename(columns={'Adj Close': 'price'})
    stocks = stocks[(stocks.index >= start) & (stocks.index < end)]

    agg_reviews = pd.DataFrame()
    agg_reviews['rating'] = reviews.rating
    agg_reviews['date'] = pd.to_datetime(reviews.index - pd.to_timedelta(7, unit='d'))
    agg_reviews = agg_reviews.groupby(pd.Grouper(key='date', freq='W-MON'))['rating'].mean().reset_index().sort_values('date').fillna(np.mean(agg_reviews.rating))
    
    rolling_reviews = reviews.rating.rolling(window=31, min_periods=1).mean()
    detrended_reviews = (reviews.rating - rolling_reviews)

    agg_stocks = pd.DataFrame()
    agg_stocks['price'] = stocks
    agg_stocks['date'] = pd.to_datetime(stocks.index - pd.to_timedelta(7, unit='d'))
    agg_stocks = agg_stocks.groupby(pd.Grouper(key='date', freq='W-MON'))['price'].last().reset_index().sort_values('date')

    rolling_stocks = agg_stocks.price.rolling(window=4, min_periods=1).mean()
    detrended_stocks = (agg_stocks.price - rolling_stocks)

    return agg_reviews, detrended_reviews, agg_stocks, detrended_stocks


def adf_test(timeseries):

    dftest = adfuller(timeseries, autolag='AIC')
    result = pd.Series(dftest[0:4], index=['Test Statistic','P-value','Lags Used','No of Observations'])
    
    for key,value in dftest[4].items():
        result['Critical Value (%s)'%key] = value

    return result


def cc_values(series1, series2):

    lags = signal.correlation_lags(len(series1), len(series2))
    p = series1
    q = series2
    p = (p - np.mean(p)) / (np.std(p) * len(p))
    q = (q - np.mean(q)) / (np.std(q))  
    c = np.correlate(p, q, 'full')

    return c, lags


def cc_plot(lags, cc, company):
    
    fig, ax = plt.subplots(figsize=(9, 6))
    ax.set_facecolor("white")
    ax.plot(lags, cc)
    ax.axhline(-2/np.sqrt(23), color='red', label='5% confidence interval')
    ax.axhline(2/np.sqrt(23), color='red')
    ax.axvline(x = 0, color = 'black', lw = 1)
    ax.axhline(y = 0, color = 'black', lw = 1)
    ax.axhline(y = np.max(cc), color = 'blue', lw = 1, 
    linestyle='--', label = 'highest +/- correlation')
    ax.axhline(y = np.min(cc), color = 'blue', lw = 1, 
    linestyle='--')
    ax.set(ylim = [-1, 1])
    ax.set_title(f'Cross Correlation: {company} Employee Reviews and Stock Price', fontsize = 15)
    ax.set_ylabel('Correlation Coefficients', 
    fontsize = 12)
    ax.set_xlabel('Time Lags', fontsize = 12)
    plt.legend()


In [ ]:
COMPANY = "Apple"

agg_reviews, detrended_reviews, agg_stocks, detrended_stocks = pre_analysis(review_df, stock_df, COMPANY, '2020-01-01', '2022-10-01')

print(adf_test(agg_reviews.rating)['P-value'])
print(adf_test(detrended_stocks)['P-value'])

In [ ]:
COMPANY = "Google"
agg_reviews, detrended_reviews, agg_stocks, detrended_stocks = pre_analysis(review_df, stock_df, COMPANY, '2020-01-01', '2022-10-01')

cc_matrix, lags = cc_values(agg_reviews.rating, detrended_stocks)
print("Maximum correlation index: ", cor_index:=np.unravel_index(np.argmax(cc_matrix, axis=None), cc_matrix.shape)[0], "\n",
        "Correlation of: ", cc_matrix[cor_index])
cc_plot(lags, cc_matrix, COMPANY)
plt.savefig(f"cross-correlation-{COMPANY}.png")

In [ ]:
COMPANY = "Apple"
agg_reviews, detrended_reviews, agg_stocks, detrended_stocks = pre_analysis(review_df, stock_df, COMPANY, '2020-01-01', '2022-10-01')

cc_matrix, lags = cc_values(agg_reviews.rating, detrended_stocks)
print("Maximum correlation index: ", cor_index:=np.unravel_index(np.argmax(cc_matrix, axis=None), cc_matrix.shape)[0], "\n",
        "Correlation of: ", cc_matrix[cor_index])
cc_plot(lags, cc_matrix, COMPANY)
plt.savefig(f"cross-correlation-{COMPANY}.png")

In [ ]:
COMPANY = "Meta"
agg_reviews, detrended_reviews, agg_stocks, detrended_stocks = pre_analysis(review_df, stock_df, COMPANY, '2020-01-01', '2022-10-01')

cc_matrix, lags = cc_values(agg_reviews.rating, detrended_stocks)
print("Maximum correlation index: ", cor_index:=np.unravel_index(np.argmax(cc_matrix, axis=None), cc_matrix.shape)[0], "\n",
        "Correlation of: ", cc_matrix[cor_index])
cc_plot(lags, cc_matrix, COMPANY)
plt.savefig(f"cross-correlation-{COMPANY}.png")

In [ ]:
COMPANY = "Amazon"
agg_reviews, detrended_reviews, agg_stocks, detrended_stocks = pre_analysis(review_df, stock_df, COMPANY, '2020-01-01', '2022-10-01')

cc_matrix, lags = cc_values(agg_reviews.rating, detrended_stocks)
print("Maximum correlation index: ", cor_index:=np.unravel_index(np.argmax(cc_matrix, axis=None), cc_matrix.shape)[0], "\n",
        "Correlation of: ", cc_matrix[cor_index])
cc_plot(lags, cc_matrix, COMPANY)
plt.savefig(f"cross-correlation-{COMPANY}.png")

In [ ]:
COMPANY = "Microsoft"
agg_reviews, detrended_reviews, agg_stocks, detrended_stocks = pre_analysis(review_df, stock_df, COMPANY, '2020-01-01', '2022-10-01')

cc_matrix, lags = cc_values(agg_reviews.rating, detrended_stocks)
print("Maximum correlation index: ", cor_index:=np.unravel_index(np.argmax(cc_matrix, axis=None), cc_matrix.shape)[0], "\n",
        "Correlation of: ", cc_matrix[cor_index])
cc_plot(lags, cc_matrix, COMPANY)
plt.savefig(f"cross-correlation-{COMPANY}.png")

In [ ]:
COMPANY = "Nvidia"
agg_reviews, detrended_reviews, agg_stocks, detrended_stocks = pre_analysis(review_df, stock_df, COMPANY, '2020-01-01', '2022-10-01')

cc_matrix, lags = cc_values(agg_reviews.rating, detrended_stocks)
print("Maximum correlation index: ", cor_index:=np.unravel_index(np.argmax(cc_matrix, axis=None), cc_matrix.shape)[0], "\n",
        "Correlation of: ", cc_matrix[cor_index])
cc_plot(lags, cc_matrix, COMPANY)
plt.savefig(f"cross-correlation-{COMPANY}.png")

In [ ]:
COMPANY = "Tesla"
agg_reviews, detrended_reviews, agg_stocks, detrended_stocks = pre_analysis(review_df, stock_df, COMPANY, '2020-01-01', '2022-10-01')

cc_matrix, lags = cc_values(agg_reviews.rating, detrended_stocks)
print("Maximum correlation index: ", cor_index:=np.unravel_index(np.argmax(cc_matrix, axis=None), cc_matrix.shape)[0], "\n",
        "Correlation of: ", cc_matrix[cor_index])
cc_plot(lags, cc_matrix, COMPANY)
plt.savefig(f"cross-correlation-{COMPANY}.png")